<a href="https://colab.research.google.com/github/sasireddyvari/Spam_Classifier/blob/master/spam_ham_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip '/content/drive/MyDrive/Colab Notebooks/Projects/SpamClassifier-master.zip' -d '/content/drive/MyDrive/Colab Notebooks/Projects/SpamCLassifier'

Archive:  /content/drive/MyDrive/Colab Notebooks/Projects/SpamClassifier-master.zip
5fa41eaf3ec0d21d3ffced673800d25b6b8ffc20
   creating: /content/drive/MyDrive/Colab Notebooks/Projects/SpamCLassifier/SpamClassifier-master/
 extracting: /content/drive/MyDrive/Colab Notebooks/Projects/SpamCLassifier/SpamClassifier-master/README.md  
  inflating: /content/drive/MyDrive/Colab Notebooks/Projects/SpamCLassifier/SpamClassifier-master/Spamclassifier.py  
   creating: /content/drive/MyDrive/Colab Notebooks/Projects/SpamCLassifier/SpamClassifier-master/smsspamcollection/
  inflating: /content/drive/MyDrive/Colab Notebooks/Projects/SpamCLassifier/SpamClassifier-master/smsspamcollection/SMSSpamCollection  
  inflating: /content/drive/MyDrive/Colab Notebooks/Projects/SpamCLassifier/SpamClassifier-master/smsspamcollection/readme  


In [9]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,TfidfTransformer
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
messages = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/SpamCLassifier/smsspamcollection/SMSSpamCollection')
messages

ParserError: ignored

In [3]:
messages = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/SpamCLassifier/smsspamcollection/SMSSpamCollection', sep='\t')
messages

,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
0,ham,Ok lar... Joking wif u oni...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,U dun say so early hor... U c already then say...
3,ham,"Nah I don't think he goes to usf, he lives aro..."
4,spam,FreeMsg Hey there darling it's been 3 week's n...
...,...,...
5566,spam,This is the 2nd time we have tried 2 contact u...
5567,ham,Will ü b going to esplanade fr home?
5568,ham,"Pity, * was in mood for that. So...any other s..."
5569,ham,The guy did some bitching but I acted like i'd...


In [4]:
messages = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/SpamCLassifier/smsspamcollection/SMSSpamCollection', sep='\t',names=["label", "message"])
messages

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [10]:
wnl = WordNetLemmatizer()
corpus=[]

for i in range(0,len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review = review.lower()
    review = review.split()
    
    review = [wnl.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)


In [11]:
corpus_df=pd.DataFrame(corpus,columns=['messages_normalised'])

df=pd.concat([messages,corpus_df],axis=1)

messages.shape
corpus_df.shape

x= df.iloc[:,-1]
y= df.iloc[:,0]

In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 0)

# Creating the Bag of Words model
from sklearn.pipeline import Pipeline
spam_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', MultinomialNB()),
                         ])
spam_clf_model = spam_clf.fit(x_train, y_train)

y_predict = spam_clf_model.predict(x_test)

from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
accuracy_score(y_predict,y_test)


0.9730941704035875

In [14]:
confusion_matrix(y_test,y_predict)

array([[955,   0],
       [ 30, 130]])

In [16]:
from sklearn.externals import joblib

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [38]:
joblib.dump(spam_clf_model,"/content/drive/MyDrive/Colab Notebooks/Projects/SpamCLassifier/spam_classifier_joblib_v1")

['/content/drive/MyDrive/Colab Notebooks/Projects/SpamCLassifier/spam_classifier_joblib_v1']

In [32]:
model=joblib.load("/content/drive/MyDrive/Colab Notebooks/Projects/SpamCLassifier/spam_classifier_joblib")

In [33]:
text=input("Enter the text:")
final_text=text.split(".")
str(model.predict(final_text))

Enter the text:Sasi you won 1,00,000 prize


"['spam']"

In [34]:
text

'Sasi you won 1,00,000 prize'

In [35]:
text.split('.')

['Sasi you won 1,00,000 prize']